Nearest Neighbour Classifier
--------------------------------

In this notebook we will perform sentiment analysis for movie reviews using KNN nearest neighbour classifier using cosine similarity among reviews.

In [18]:
import pandas as pd
import numpy as np
import scipy.sparse as sp
import re
import nltk
import utility as util
from sklearn.metrics.pairwise import cosine_similarity
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import  CountVectorizer

We will be reading training and test data set and store in pandas dataframe as train and test with columns polarity and review for train.

In [ ]:
with open('./train.dat','r') as f:
    #next(f) # skip first row
    train = pd.DataFrame(line.split('\t') for line in f.readlines())
train.columns = ['polarity', 'review']

with open('./Test/test.dat','r') as f:
    #next(f) # skip first row
    test = pd.DataFrame(line for line in f.readlines())
test.columns = ['review']

To get same dimensions of csr_matrix for train and test we will be merging train and test reviews columns.

In [19]:
data = []
for rev in train['review']:
     data.append(rev)
for rev in test['review']:
     data.append(rev)

data = np.array(data)

Data set contains html artifacts and some uncessary data which might effect classification accuracy. To overcome this we preprocess data and remove html tags and other character from reviews. When we analyse review they are many common or popular words which don't contribute to polarity of reviews. We merged each reviews and calculated frequency of each words, and added some words to our stop words list. Using nltk stop_words and our stop words list, we have filterd out popular words from each review.

In [20]:
data = util.preprocess(data)

After preprocesing data, it return list of words in each review with there frequency. We further remove words less than 4 and words with frequency greater than 5 from our reviews.

In [21]:
def filterLen(docs, minlen):
    r""" filter out terms that are too short. 
    docs is a list of lists, each inner list is a document represented as a list of words
    minlen is the minimum length of the word to keep
    """
    return [ [t.lower() for t,v in d.items() if len(t) >= minlen and v < 6] for d in docs ]

data = filterLen(data ,4)

These our the function which we will use to build csr matrix, normalize them , view csr matrix info. We will pass our review data to build a normalize csr_matrix.

In [22]:
def build_matrix(docs):
    r""" Build sparse matrix from a list of documents, 
    each of which is a list of word/terms in the document.  
    """
    nrows = len(docs)
    idx = {}
    tid = 0
    nnz = 0
    for d in docs:
        nnz += len(set(d))
        for w in d:
            if w not in idx:
                idx[w] = tid
                tid += 1
    ncols = len(idx)
        
    # set up memory
    ind = np.zeros(nnz, dtype=np.int)
    val = np.zeros(nnz, dtype=np.double)
    ptr = np.zeros(nrows+1, dtype=np.int)
    i = 0  # document ID / row counter
    n = 0  # non-zero counter
    # transfer values
    for d in docs:
        cnt = Counter(d)
        keys = list(k for k,_ in cnt.most_common())
        l = len(keys)
        for j,k in enumerate(keys):
            ind[j+n] = idx[k]
            val[j+n] = cnt[k]
        ptr[i+1] = ptr[i] + l
        n += l
        i += 1
            
    mat = csr_matrix((val, ind, ptr), shape=(nrows, ncols), dtype=np.double)
    mat.sort_indices()
    
    return mat

def csr_info(mat, name="", non_empy=False):
    r""" Print out info about this CSR matrix. If non_empy, 
    report number of non-empty rows and cols as well
    """
    if non_empy:
        print("%s [nrows %d (%d non-empty), ncols %d (%d non-empty), nnz %d]" % (
                name, mat.shape[0], 
                sum(1 if mat.indptr[i+1] > mat.indptr[i] else 0 
                for i in range(mat.shape[0])), 
                mat.shape[1], len(np.unique(mat.indices)), 
                len(mat.data)))
    else:
        print( "%s [nrows %d, ncols %d, nnz %d]" % (name, 
                mat.shape[0], mat.shape[1], len(mat.data)) )

def csr_l2normalize(mat, copy=False, **kargs):
    r""" Normalize the rows of a CSR matrix by their L-2 norm. 
    If copy is True, returns a copy of the normalized matrix.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # normalize
    for i in range(nrows):
        rsum = 0.0    
        for j in range(ptr[i], ptr[i+1]):
            rsum += val[j]**2
        if rsum == 0.0:
            continue  # do not normalize empty rows
        rsum = 1.0/np.sqrt(rsum)
        for j in range(ptr[i], ptr[i+1]):
            val[j] *= rsum
            
    if copy is True:
        return mat

def csr_idf(mat, copy=False, **kargs):
    r""" Scale a CSR matrix by idf. 
    Returns scaling factors as dict. If copy is True, 
    returns scaled matrix and scaling factors.
    """
    if copy is True:
        mat = mat.copy()
    nrows = mat.shape[0]
    nnz = mat.nnz
    ind, val, ptr = mat.indices, mat.data, mat.indptr
    # document frequency
    df = defaultdict(int)
    for i in ind:
        df[i] += 1
    # inverse document frequency
    for k,v in df.items():
        df[k] = np.log(nrows / float(v))  ## df turns to idf - reusing memory
    # scale by idf
    for i in range(0, nnz):
        val[i] *= df[ind[i]]
        
    return df if copy is False else mat

sparse_matrix = build_matrix(data)
csr_info(sparse_matrix)


sparse_matrix = csr_idf(sparse_matrix, copy=True)
sparse_matrix = csr_l2normalize(sparse_matrix, copy=True)

After building csr matrix, we will divide it into train_mat and test_mat, so that our matrix our of same dimensions , which is required for matrix multiplication.

In [23]:
train_mat = sparse_matrix[0:25000]
test_mat =  sparse_matrix[25000:]

 [nrows 50000, ncols 92418, nnz 3373670]


Now we will calculate cosine similarity between train_mat and test_mat.Due to machine incompetency, we use batch size of 10 to calculate similarity.

In [24]:
def cosine_similarity_n_space(m1, m2, batch_size=10):
    assert m1.shape[1] == m2.shape[1]
    ret = np.ndarray((m1.shape[0], m2.shape[0]))
    for row_i in range(0, int(m1.shape[0] / batch_size) + 1):
        start = row_i * batch_size
        end = min([(row_i + 1) * batch_size, m1.shape[0]])
        if end <= start:
            break # cause I'm too lazy to elegantly handle edge cases
        rows = m1[start: end]
        sim = cosine_similarity(rows, m2) # rows is O(1) size
        ret[start: end] = sim
    return ret

cosineSimilarityValue = cosine_similarity_n_space(test_mat,train_mat)

In our csr_mat each row represents test review which can be compared by each column represented by train reviews. We will calculate max K values for each train review (KNN) and map there index in train data frame to get there polarity.If count of negative polarity is greater than postive a "-1" label will be assigned to review otherwise "+1". 

In [26]:
f = open('./format.dat', 'w')
count = 0
# for row in cosineSimilarityValue:

#     k=93
#    # similar_index = np.argsort(row)[::-1][:k]

for row in cosineSimilarityValue:

    k=220
    partitioned = np.argpartition(-row, k)  
    similar_index = partitioned[:k]
    
    negative = 0
    positive = 0

    for index in similar_index:

        if train['polarity'][index] == '-1':
            negative+=1
        elif train['polarity'][index] == '+1':
            positive+=1
            
    
    if negative > positive:
        f.write('-1\n')
        count+=1
    else:
        f.write('+1\n')
        count+=1

print("Total lines : %d"%count)

Total lines : 25000


In [11]:
print("Total lines : %d"%count)

Total lines : 25000
